# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-uq1w5jgv
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-uq1w5jgv
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit 97049d67d83ec6129569d442bd365c7a5e490578
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-02-04 13:59:31--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23304 (23K) [text/plain]
Saving to: ‘structured_qa.csv.1’

structured_qa.csv.1 100%[===================>]  22.76K  --.-KB/s    in 0.002s  

2025-02-04 13:59:32 (10.6 MB/s) - ‘structured_qa.csv.1’ saved [23304/23304]



# Setup

In [3]:
import os
import google.generativeai as genai
from google.colab.userdata import get, SecretNotFoundError

try:
    genai.configure(api_key=get("GOOGLE_API_KEY"))
except SecretNotFoundError as e:
    raise RuntimeError("Please set the GOOGLE_API_KEY secret to your API key") from e
os.environ["LOGURU_LEVEL"] = "INFO"

In [4]:
from loguru import logger

## Function to Process all questions for a single Document

In [5]:
import json
import time


def process_document_questions(
    document_file,
    document_data,
    model,
):
    logger.info("Uploading file")
    file = genai.upload_file(document_file, mime_type="application/pdf")
    while file.state.name == "PROCESSING":
        logger.debug("Waiting for file to be processed.")
        time.sleep(2)
        file = genai.get_file(file.name)

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        if model.n > 0 and model.n % 9 == 0:
            logger.info("Waiting for 60 seconds")
            time.sleep(60)
        question = row["question"]
        logger.info(f"Question: {question}")
        try:
            response = model.model.generate_content([file, question])
        except Exception:
            answers[index] = "Error"
            sections[index] = None
            continue
        logger.info(response.text)
        answers[index] = response.text
        sections[index] = None
        model.n += 1
    return answers, sections

## Load Model

In [6]:
from structured_qa.model_loaders import load_gemini_model

In [7]:
SYSTEM_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""

In [8]:
model = load_gemini_model("gemini-2.0-flash-exp", system_prompt=SYSTEM_PROMPT)
model.n = 0

# Run Benchmark

In [9]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd


logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document_questions(
        downloaded_document, document_data, model
    )

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 13:59:35.081 | INFO     | __main__:<cell line: 0>:7 - Loading input data
2025-02-04 13:59:35.093 | INFO     | __main__:<cell line: 0>:13 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-02-04 13:59:35.096 | INFO     | __main__:<cell line: 0>:19 - File HAI_AI-Index-Report-2024.pdf.pdf already exists
2025-02-04 13:59:35.098 | INFO     | __main__:process_document_questions:10 - Uploading file
2025-02-04 13:59:37.120 | INFO     | __main__:process_document_questions:17 - Predicting
2025-02-04 13:59:37.122 | INFO     | __main__:process_document_questions:25 - Question: which type of risk was identified as the leading concern globally? -A: Fairness risks. -B: Privacy and data governance risks. -C: Risks related to generative AI deployment.
2025-02-04 14:00:37.504 | INFO     | __main__:process_document_questions:32 - B
2025-02-04 14:00:37.507 | INFO     | __main__:process_document_questions:25 - Question: In which geogr

# Results

In [12]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
26,26,https://authorsalliance.org/wp-content/uploads...,Techincal Documentation,CHAPTER 5: WHERE DO YOU WANT TO MAKE YOUR WORK...,Are Gold Open Access and Green Open Access mut...,NO,YES,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
34,34,https://arxiv.org/pdf/2201.11903,Scientific Report,3.2 Results,How many random samples were examined to under...,100,50,NaN
68,68,https://docs.nvidia.com/cuda/pdf/CUDA_C_Progra...,Techincal Documentation,5.2. Thread Hierarchy,Can you identify a thread with a four-dimensio...,NO,I NEED MORE INFO,NaN
78,78,https://docs.nvidia.com/cuda/pdf/CUDA_C_Progra...,Techincal Documentation,23.1. What is Lazy Loading?,Can you enable lazy loading by setting the env...,NO,YES,NaN
90,90,https://arxiv.org/pdf/2302.13971,Scientific Report,2.1 Pre-training Data,How many languages did the Wikipedia data cover?,20,NUMBER,NaN


In [13]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.941747572815534